<a href="https://colab.research.google.com/github/kartik-5479/Gym_management_system/blob/main/GYM_MANAGEMENT_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Paste & run this in Google Colab. You'll be prompted to paste your ngrok authtoken (hidden).

# Install required packages
!pip install -q flask pyngrok

import os, csv, hashlib, hmac, re, getpass, threading, time
from datetime import datetime, timezone, timedelta
from typing import Optional

# Securely read authtoken from user (hidden input)
token = getpass.getpass("Paste your ngrok authtoken (it will be hidden): ").strip()
if not token:
    raise SystemExit("No authtoken provided — run again and paste your ngrok authtoken.")

# Configure pyngrok
from pyngrok import ngrok
ngrok.set_auth_token(token)
print("ngrok authtoken set. Starting Flask + ngrok...")

# --------------------------- Backend: your project code ---------------------------
DATA_DIR = '/content/gym_data'  # change to '/content/drive/MyDrive/gym_data' to persist to Drive
MEMBERS_FILE = os.path.join(DATA_DIR, 'members.csv')
ATTENDANCE_FILE = os.path.join(DATA_DIR, 'attendance.csv')

HASH_NAME = 'sha256'
ITERATIONS = 100_000
SALT_PREFIX = b'GYM-SALT-'
IST = timezone(timedelta(hours=5, minutes=30))

def ensure_data_dir():
    os.makedirs(DATA_DIR, exist_ok=True)

def _clean_phone(phone: str) -> str:
    return re.sub(r"\D", "", phone)

def readable_member_id(name: str, phone: str, dob: str) -> str:
    parts = [p for p in name.strip().split() if p]
    initials = 'X' if not parts else ''.join([p[0] for p in parts[:3]]).upper()
    digits = _clean_phone(phone)
    last4 = digits[-4:] if len(digits) >= 4 else digits.zfill(4)
    try:
        dt = datetime.strptime(dob.strip(), '%Y-%m-%d')
        dob_part = dt.strftime('%y%m%d')
    except Exception:
        digits_dob = re.sub(r"\D", "", dob)
        dob_part = digits_dob[-6:].zfill(6)
    return f"{initials}-{last4}-{dob_part}"

def hash_password(password: str, salt_material: str) -> str:
    salt = SALT_PREFIX + salt_material.encode('utf-8')
    dk = hashlib.pbkdf2_hmac(HASH_NAME, password.encode('utf-8'), salt, ITERATIONS)
    return dk.hex()

def verify_password(stored_hash: str, password_attempt: str, salt_material: str) -> bool:
    attempt_hash = hash_password(password_attempt, salt_material)
    return hmac.compare_digest(stored_hash, attempt_hash)

def init_files():
    ensure_data_dir()
    if not os.path.exists(MEMBERS_FILE):
        with open(MEMBERS_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['id', 'name', 'phone', 'dob', 'email', 'password_hash', 'joined_on'])
    if not os.path.exists(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['timestamp', 'member_id', 'name', 'entry_type', 'notes'])

def read_members() -> dict:
    d = {}
    if not os.path.exists(MEMBERS_FILE):
        return d
    with open(MEMBERS_FILE, 'r', encoding='utf-8') as f:
        r = csv.DictReader(f)
        for row in r:
            d[row['id']] = row
    return d

def write_members(members: dict):
    with open(MEMBERS_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['id', 'name', 'phone', 'dob', 'email', 'password_hash', 'joined_on'])
        for mid, info in members.items():
            w.writerow([mid, info.get('name',''), info.get('phone',''), info.get('dob',''), info.get('email',''), info.get('password_hash',''), info.get('joined_on','')])

def add_member(name: str, phone: str, dob: str, email: str, password: str) -> str:
    member_id = readable_member_id(name, phone, dob)
    joined_on = datetime.now(IST).isoformat()
    password_hash = hash_password(password, member_id)

    members = read_members()
    if member_id in members:
        existing = members[member_id]
        if existing['name'].strip().lower() == name.strip().lower() and _clean_phone(existing['phone']) == _clean_phone(phone):
            members[member_id].update({'name': name, 'phone': phone, 'dob': dob, 'email': email})
            write_members(members)
            return member_id
        import hashlib as _hl
        suffix = _hl.sha256((name+phone+dob).encode('utf-8')).hexdigest()[-3:].upper()
        member_id = f"{member_id}-{suffix}"

    with open(MEMBERS_FILE, 'a', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow([member_id, name, phone, dob, email, password_hash, joined_on])
    return member_id

def log_attendance(member_id: str, name: str, entry_type: str='IN', notes: str=''):
    ts = datetime.now(IST).isoformat()
    with open(ATTENDANCE_FILE, 'a', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow([ts, member_id, name, entry_type, notes])

def read_attendance() -> list:
    if not os.path.exists(ATTENDANCE_FILE):
        return []
    with open(ATTENDANCE_FILE, 'r', encoding='utf-8') as f:
        return list(csv.DictReader(f))

init_files()

# --------------------------- Flask app (uses ngrok) ---------------------------
from flask import Flask, request, redirect, url_for, render_template_string, flash

app = Flask(__name__)
app.secret_key = 'dev-key-for-demo'  # change for production

HOME_HTML = """
<!doctype html>
<h1>Gym Attendance Portal (Flask)</h1>
<p>Timestamps shown are in IST (UTC+5:30).</p>
<ul>
  <li><a href="/register">Register Member</a></li>
  <li><a href="/login">Member Entry (Login)</a></li>
  <li><a href="/exit">Member Exit</a></li>
  <li><a href="/members">View Members</a></li>
  <li><a href="/attendance">View Attendance</a></li>
</ul>
"""

REGISTER_HTML = """
<h2>Register New Member</h2>
<form method="post">
  Full name:<br><input name="name" required><br><br>
  Phone (digits):<br><input name="phone" required><br><br>
  DOB (YYYY-MM-DD):<br><input name="dob" required><br><br>
  Email (optional):<br><input name="email"><br><br>
  Password:<br><input name="password" type="password" required><br><br>
  <button type="submit">Register</button>
</form>
<p><a href="/">Back</a></p>
"""

LOGIN_HTML = """
<h2>Member Entry (Login)</h2>
<form method="post">
  Member ID:<br><input name="member_id" required><br><br>
  Password:<br><input name="password" type="password" required><br><br>
  <button type="submit">Enter (Log IN)</button>
</form>
<p><a href="/">Back</a></p>
"""

EXIT_HTML = """
<h2>Member Exit</h2>
<form method="post">
  Member ID:<br><input name="member_id" required><br><br>
  Password:<br><input name="password" type="password" required><br><br>
  <button type="submit">Exit (Log OUT)</button>
</form>
<p><a href="/">Back</a></p>
"""

TABLE_HTML = """
<h2>{{title}}</h2>
<table border="1" cellpadding="6" cellspacing="0">
  <thead><tr>{% for h in headers %}<th>{{h}}</th>{% endfor %}</tr></thead>
  <tbody>
  {% for row in rows %}
    <tr>{% for h in headers %}<td>{{row[h]}}</td>{% endfor %}</tr>
  {% endfor %}
  </tbody>
</table>
<p><a href="/">Back</a></p>
"""

from flask import render_template_string

@app.route('/')
def home():
    return HOME_HTML

@app.route('/register', methods=['GET','POST'])
def register():
    if request.method == 'GET':
        return REGISTER_HTML
    name = request.form.get('name','').strip()
    phone = request.form.get('phone','').strip()
    dob = request.form.get('dob','').strip()
    email = request.form.get('email','').strip()
    password = request.form.get('password','').strip()
    if not (name and phone and dob and password):
        flash("Please fill required fields")
        return redirect(url_for('register'))
    mid = add_member(name, phone, dob, email, password)
    flash(f"Registered {name} with ID: {mid}")
    return redirect(url_for('register'))

@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'GET':
        return LOGIN_HTML
    mid = request.form.get('member_id','').strip().upper()
    password = request.form.get('password','').strip()
    members = read_members()
    if mid not in members:
        flash("Member ID not found")
        return redirect(url_for('login'))
    if verify_password(members[mid]['password_hash'], password, mid):
        log_attendance(mid, members[mid]['name'], 'IN', notes='Web-entry')
        flash(f"Entry logged for {members[mid]['name']} ({mid})")
    else:
        log_attendance(mid, members[mid]['name'], 'FAILED', notes='Bad password')
        flash("Authentication failed")
    return redirect(url_for('login'))

@app.route('/exit', methods=['GET','POST'])
def exit_member():
    if request.method == 'GET':
        return EXIT_HTML
    mid = request.form.get('member_id','').strip().upper()
    password = request.form.get('password','').strip()
    members = read_members()
    if mid not in members:
        flash("Member ID not found")
        return redirect(url_for('exit_member'))
    if verify_password(members[mid]['password_hash'], password, mid):
        log_attendance(mid, members[mid]['name'], 'OUT', notes='Web-exit')
        flash(f"Exit logged for {members[mid]['name']} ({mid})")
    else:
        log_attendance(mid, members[mid]['name'], 'FAILED_EXIT', notes='Bad password')
        flash("Authentication failed")
    return redirect(url_for('exit_member'))

@app.route('/members')
def members_page():
    members = list(read_members().values())
    headers = ['id','name','phone','dob','email','joined_on']
    return render_template_string(TABLE_HTML, title="Registered Members", headers=headers, rows=members)

@app.route('/attendance')
def attendance_page():
    rows = read_attendance()
    headers = ['timestamp','member_id','name','entry_type','notes']
    rows = list(reversed(rows))
    return render_template_string(TABLE_HTML, title="Attendance Log (latest first)", headers=headers, rows=rows)

def start_tunnel_and_app():
    public_url = ngrok.connect(5000).public_url
    print("Public URL:", public_url)
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=start_tunnel_and_app, daemon=True)
thread.start()
time.sleep(3)
print("If you see 'Public URL:' above, open it to use the web UI.")



Paste your ngrok authtoken (it will be hidden): ··········
ngrok authtoken set. Starting Flask + ngrok...
Public URL: https://inscriptive-placably-katie.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


If you see 'Public URL:' above, open it to use the web UI.
